In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
import plotly.express as px
import optuna


## Carregar os Dados

In [2]:
df_clientes = pd.read_csv('./data/dataset_clientes_pj.csv')

In [3]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [4]:
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


## EDA

In [5]:
percentual_inovacao = df_clientes.value_counts('inovacao')/len(df_clientes) * 100
px.bar(percentual_inovacao,color=percentual_inovacao.index)

# Teste ANOVA (Análise de Variância)
 Verificar se há variações significativas na média de faturamento mensal para diferentes níveis de inovação
 Suposições / Pressupostos:
 - Observações independentes
 - Variável dependente é contínua 
 - Segue uma distribuição normal
 - Homogeneidade das variâncias
 - Amostras sejam de tamanhos iguais



In [6]:
# Checar se as variâncias (faturamento) entre os grupos (inovação) são homogêneas
# Aplicar Teste de Bartlett
# H0 - Variâncias são iguais
# H1 - Variâncias não são iguais

from scipy.stats import bartlett

# Separando os dados de faturamento em grupos com base na coluna 'inovacao'
dados_agrupados = [df_clientes['faturamento_mensal'][df_clientes['inovacao'] == grupo] for grupo in df_clientes['inovacao'].unique()]

dados_agrupados

[0      713109.95
 6      771179.95
 8      888983.66
 25    1237763.98
 29     489936.94
 44     973809.28
 51     296714.92
 57    2263343.91
 62     866778.96
 69     662659.12
 71    1191112.82
 72     936929.11
 87    1124033.51
 93    1081112.62
 100   1005847.92
 101    909973.14
 132    949003.35
 142    831586.64
 166    103994.80
 167   1236114.44
 187    494926.48
 194    174738.56
 196   1376121.85
 203   1248640.73
 207   1032819.03
 212   1194723.40
 218    608858.82
 242   2390677.22
 253    911615.28
 290    908289.25
 292   2107306.42
 299   1212901.40
 303   1177227.98
 304    499441.30
 312   1094337.38
 313    880090.72
 319    854382.67
 328    823374.93
 331   1006625.13
 343   1569380.97
 366    629572.60
 371   1843761.24
 392    715243.69
 396    989883.92
 403   1195575.01
 408    603305.03
 409   1220941.70
 424    725022.65
 432    844782.42
 438   1021085.99
 450    475032.23
 452   1042344.43
 454   1198949.59
 465    805517.39
 486    445571.87
 489    41

In [7]:
# Executar o teste de Bartlett
bartlett_test_statistic, bartlett_p_value = bartlett(*dados_agrupados)

print(f'Estatistica do teste de Bartlett: {bartlett_test_statistic}')
print(f'P_value do teste de Bartlett: {bartlett_p_value}')

Estatistica do teste de Bartlett: 10.901203117231173
P_value do teste de Bartlett: 0.28254182954905804


In [8]:
# Executar o Teste de Shapiro-Wilk
# Verificar se os dados seguem uma distribuição normal
# H0 - Segue uma distribuição normal
# H1 - Não segue uma distribuição normal

# Executar o teste

from scipy.stats import shapiro

shapiro_test_statistic, shapiro_p_value = shapiro(df_clientes['faturamento_mensal'])

print(f'Estatistica do teste de Shapiro: {shapiro_test_statistic}')
print(f'P_value do teste de Shapiro: {shapiro_p_value}')

Estatistica do teste de Shapiro: 0.9959857602472711
P_value do teste de Shapiro: 0.23513451034389005


In [9]:
# Aplicar a ANOVA de Welch, pois as amostras são de tamanhos diferentes
# H0 - Não há diferenças significativas entre as médias dos grupos
# H1 - Há pelo menos uma diferença significativa entre as médias dos grupos

from pingouin import welch_anova

aov = welch_anova(dv='faturamento_mensal', between='inovacao',data=df_clientes)

# Exibindo os resultados
print(f'Estatistica do teste de ANOVA Welsh: {aov.loc[0,'F']}')
print(f'P_value do teste de ANOVA Welsh: {aov.loc[0,'p-unc']}')

Estatistica do teste de ANOVA Welsh: 1.1269836194061693
P_value do teste de ANOVA Welsh: 0.34526211273912577


In [10]:
aov

,Source,ddof1,ddof2,F,p-unc,np2
0,inovacao,9,197.74,1.13,0.35,0.02


## Treinar o Algoritmo K-Means

In [11]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [12]:
# Selecionar as colunas para clusterização
X = df_clientes.copy()

# Separação entre as variáveis númericas, categóricas e ordiais
numeric_features = ['faturamento_mensal','numero_de_funcionarios','idade']
categorical_features = ['localizacao','atividade_economica']
ordinal_features = ['inovacao']

# Aplciar Transformações por tipo
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()


In [15]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer,ordinal_features)
    ]
)

X_tranformed = preprocessor.fit_transform(X)

X_tranformed

array([[-0.74634498, -0.54179191, -1.10058849, ...,  0.        ,
         0.        ,  1.        ],
       [-0.56165548, -1.5035527 ,  1.94344851, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40582654,  1.06114274, -1.77704115, ...,  0.        ,
         0.        ,  9.        ],
       ...,
       [ 2.8196246 , -1.18296577,  0.25231684, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.03321411, -0.54179191, -1.43881482, ...,  0.        ,
         0.        ,  3.        ],
       [-2.03011486, -0.22120498, -1.77704115, ...,  1.        ,
         0.        ,  9.        ]], shape=(500, 12))

In [17]:
# Optuna para Otimização de Hiperparâmetros
def kmeans_objective(trial):
    # Definir os hiperparâmetros a serem ajustados
    n_clusters = trial.suggest_int('n_clusters',3,10)
    distance_metric = trial.suggest_categorical('distance_metric', ['euclidean', 'minkowski'])

    # Criar o modelo
    modelo_kmeans = KMeans(n_clusters=n_clusters, random_state=51)

    # Treinar o modelo
    modelo_kmeans.fit(X_tranformed)

    # Calculado o silhouette score
    distances = pairwise_distances(X_tranformed, metric=distance_metric)
    silhouette_avg = silhouette_score(distances, modelo_kmeans.labels_)

    return silhouette_avg


In [18]:
# Criar um estudo do Optuna 
search_space = {'n_clusters':[3,4,5,6,7,8,9,10], 'distance_metric': ['euclidean', 'minkowski']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_kmeans = optuna.create_study(direction='maximize', sampler=sampler)

# Rodar o estudo
estudo_kmeans.optimize(kmeans_objective, n_trials=100)

[I 2025-12-20 09:35:26,116] A new study created in memory with name: no-name-67869ceb-38d2-4a26-98ba-8d0045b247cf
[I 2025-12-20 09:35:26,268] Trial 0 finished with value: 0.38478794965029234 and parameters: {'n_clusters': 4, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.38478794965029234.
[I 2025-12-20 09:35:26,284] Trial 1 finished with value: 0.11230048381589083 and parameters: {'n_clusters': 9, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.38478794965029234.
[I 2025-12-20 09:35:26,295] Trial 2 finished with value: 0.4445458290999088 and parameters: {'n_clusters': 3, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.4445458290999088.
[I 2025-12-20 09:35:26,306] Trial 3 finished with value: 0.38478794965029234 and parameters: {'n_clusters': 4, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.4445458290999088.
[I 2025-12-20 09:35:26,324] Trial 4 finished with value: 0.14731572416665722 and parameters: {'n_clusters': 8, 'dista

In [21]:
# Melhor Configuração encontrada pelo Optuna
best_params = estudo_kmeans.best_params

# Instanciando o modelo KMeans com melhores parâmetros
best_kmeans = KMeans(n_clusters=best_params['n_clusters'], random_state=51)
best_kmeans.fit(X_tranformed)

# Calculado o silhouette score
distances = pairwise_distances(X_tranformed, metric=best_params['distance_metric'])
best_silhouette = silhouette_score(distances, best_kmeans.labels_)

print(f'k (Número de Clusters): {best_params['n_clusters']}')
print(f'Métrica de distância selecionada: {best_params['distance_metric']}')
print(f'Silhouette Score: {best_silhouette}')



k (Número de Clusters): 3
Métrica de distância selecionada: minkowski
Silhouette Score: 0.4445458290999088


In [ ]:
# Criar uma coluna com o cluster escolhido
df_clientes['cluster'] = best_kmeans.labels_

In [23]:
# Visualizar os primeiros registros
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


## Visualizar Resultados

In [24]:
# Cruzar idade e faturamento, apresentando os clusters
px.scatter(df_clientes,x='idade',y='faturamento_mensal',color='cluster')

In [25]:
# Cruzar inovação e faturamento, apresentando os clusters
px.scatter(df_clientes,x='inovacao',y='faturamento_mensal',color='cluster')

 Percebemos que a idade não foi relevante para definir um cluster para uma empresa, porém a inovação foi determinante para definir esse cluster

In [26]:
# Cruzar numero de funcionarios e faturamento, apresentando os clusters
px.scatter(df_clientes,x='numero_de_funcionarios',y='faturamento_mensal',color='cluster')

## Salvar o modelo e o pipeline de transformação

In [ ]:
import joblib
# Salvar o modleo e o pipeline
joblib.dump(best_kmeans, 'modelo_clusterizacao_clientes.pkl')

joblib.dump(preprocessor, 'pipeline_clusterizacao.pkl')

['pipeline_clusterizacao.pkl']

## Aplicação Batch no Gradio

In [28]:
import gradio as gr

modelo = joblib.load('./modelo_clusterizacao_clientes.pkl')
pipeline = joblib.load('./pipeline_clusterizacao.pkl')


def clustering(arquivo):

    df_empresas = pd.read_csv(arquivo.name)

    # Transformar os dados do DF para o formato que o KMeans precisa
    X_tranformed = pipeline.fit_transform(df_empresas)

    # Treinar modelo
    modelo.fit(X_tranformed)

    # Criar a coluna cluster no DF
    df_empresas['cluster'] = modelo.labels_
    df_empresas.to_csv('./clusters.csv', index=False)

    return './clusters.csv'




In [30]:
# Criar a interface

app = gr.Interface(
    clustering,
    gr.File(file_types=['.csv']),
    "file"
)

# Rodar a Aplicação
app.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
